Taking the L1 data and putting it into a text file. Replace newline characters with a space

In [1]:
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

In [2]:
role = get_execution_role()

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket() 

In [23]:
df = pd.read_csv("s3://training-bert/corpus.csv", nrows=100)

In [25]:
df.content

0     It is rather hard to write with eclipse glasse...
1     As our nation celebrated Labor Day, giving att...
2     We hear quite a bit about survivors’ guilt the...
3     The study of the Hebrew language did not come ...
4     Social media is blowing up about a culture of ...
5     Many preachers took up the Isaiah 55 passage t...
6     A trip to the cardiologist is rarely routine. ...
7     **The author has been participating with QC Fa...
8     Enderly Park is blistering under an unseasonab...
9     Enderly Park is blistering under an unseasonab...
10    By Nell Green Glory to God in the highest heav...
11    Enderly Park is blistering under an unseasonab...
12    “If we have no peace, it is because we have fo...
13    By Greg and Helms Jarrell Chinua Achebe’s clas...
14    Can we feel that there are human beings, very ...
15    Flowing like a pink river, scores of women in ...
16    Enderly Park is blistering under an unseasonab...
17    Enderly Park is blistering under an unseas

In [31]:
# Concatenate these results into one file
s = df.content.str.cat(sep=' ')
s = s.replace('\n',' ')

In [34]:
s

'It is rather hard to write with eclipse glasses on this morning, but “I want to be ready,” as the old spiritual intones. Not ever known for culinary process, I even tried to turn Oreos into eclipse cookies. Massive fail!  Like many others, I have anticipated this day with a sense of foreboding and wonder. The foreboding is a lot less than the wonder, to be sure. The foreboding in my case has to do with religious quacks that claim a natural scientific event is an act of God’s judgment or an apocalyptic event signaling the end of time. That this portent, which happens quite regularly, by the way, has to haul this fake theological freight creates a stumbling block to coherent belief. It is a responsible perspective to hold a worldview that attends to the wisdom of both theology and science.  Another of these celestial alignments will happen in 2024, and I suspect many of the same arguments will be in vogue. It will be an election year, so beware! Some will proclaim that God is judging a 

In [27]:
!touch testfile.txt

In [35]:
file = open("testfile.txt","w") 
 
file.write(s) 
 
file.close() 

In [5]:
# Upload to S3

input_data = sagemaker_session.upload_data(path="testfile.txt", bucket=bucket, key_prefix="L1")

# Now doing languge modeling / pretraining

!python pretraining.py 

--output_dir "./pretrain_outputs/BERTtext"  
--train_data_file "testfile.txt"  
--seed 42  
--model_type "bert"  
--mlm  
--model_name_or_path "bert-base-uncased"  
--block_size=512  
--learning_rate 1e-4  
--num_train_epochs 5  
--do_train  
--per_gpu_train_batch_size 4   

In [27]:
!python pretraining.py --output_dir "./pretrain_outputs/BERTtext"  --train_data_file "testfile.txt" --seed 42 --model_type "bert" --mlm True --model_name_or_path "bert-base-uncased" --block_size=512 --learning_rate 1e-4 --num_train_epochs 5 --do_train --evaluate_during_training --per_gpu_train_batch_size 4 --save_total_limit 2  --save_steps 2000 

03/07/2020 21:58:31 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
model_class: <class 'transformers.modeling_bert.BertForMaskedLM'>
03/07/2020 21:58:31 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ec2-user/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
03/07/2020 21:58:31 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range":

In [41]:
!python run_pretraining.py  --output_dir "./pretrain_outputs/BERTtext" 
    --model_type bert     
    --mlm     
    --tokenizer_name ./models/BertTokenizer    
    --do_train     
    --learning_rate 1e-4     
    --num_train_epochs 5     
    --save_total_limit 2     
    --save_steps 2000     
    --per_gpu_train_batch_size 4     
    --evaluate_during_training     
    --seed 42 
    --train_data_file testfile.txt

SyntaxError: invalid syntax (<ipython-input-41-63c492649e94>, line 2)

In [3]:
# data path for the SageMaker PyTorch container. We don't need to create an own container. 
# container_data_dir = '/opt/ml/input/data/training'
# container_model_dir = '/opt/ml/model'

parameters = {
#     'train': container_data_dir,
#     'model_dir' container_model_dir: ,
    'model_type' : "bert",
    "model_name_or_path" : "bert-base-uncased",
    'mlm': True,
    'do_train': True,
    'seed' : 42,
    'num_train_epochs' : 2,
    'learning_rate' : 1e-4,
    'per_gpu_train_batch_size': 4,
    'evaluate_during_training' : False
}

In [6]:
# Amazon SageMaker PyTorch framework

train_instance_type = 'ml.p3.2xlarge'

estimator = PyTorch(entry_point='pretraining.py',
                    source_dir = './train_scripts/', # the local directory stores all relevant scripts for modeling
                    hyperparameters=parameters,
                    role=role,
                    framework_version='1.3.1',
                    train_instance_count=2,
                    train_instance_type=train_instance_type
                   )

In [7]:
estimator.fit({'train': 's3://sagemaker-us-west-2-496641494145/L1'})

2020-03-08 06:10:21 Starting - Starting the training job...
2020-03-08 06:10:23 Starting - Launching requested ML instances......
2020-03-08 06:11:22 Starting - Preparing the instances for training.........
2020-03-08 06:13:17 Downloading - Downloading input data
2020-03-08 06:13:17 Training - Downloading the training image.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-08 06:14:22,608 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-08 06:14:22,634 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-08 06:14:35,501 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-08 06:14:35,526 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
